In [1]:
# Multiple_Linear_Regression_student_performance_21.ipynb
# File name: Student_Performance
# File source: kaggle

# Cell No. 1

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import  mean_squared_error, mean_absolute_error

# Read or laod the dataset from the data file into a Pandas dataframe
data = pd.read_csv('Student_Performance.csv')

print("Performming some most basic iexploratory data analysis (EDA):")
print("The first five rows of the dataset:")
print(data.head())

n = data.shape[1]-1         # number of independent features
print("\nThe total number of features in the dataset:", n+1)
print("\nThe column titles (feature names) of the dataset:\n")
print(data.columns)

print("\nPrinting some basic information about the dataset:")
data.info()
print("\nPrinting some basic description about the dataset:")
data.describe()

# n denotes number of input/independent features, (here n=5)
# Independent features: Hours Studied, Previous Scores, Extra-curricular
# Activities, Sleep Hours, Sample Question Papers Practiced
# Dependent feature: Performance Index

Performming some most basic iexploratory data analysis (EDA):
The first five rows of the dataset:
   Hours Studied  Previous Scores Extracurricular Activities  Sleep Hours  \
0              7               99                        Yes            9   
1              4               82                         No            4   
2              8               51                        Yes            7   
3              5               52                        Yes            5   
4              7               75                         No            8   

   Sample Question Papers Practiced  Performance Index  
0                                 1               91.0  
1                                 2               65.0  
2                                 2               45.0  
3                                 2               36.0  
4                                 5               66.0  

The total number of features in the dataset: 6

The column titles (feature names) of the dataset

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced,Performance Index
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4.992900,69.445700,6.530600,4.583300,55.224800
std,2.589309,17.343152,1.695863,2.867348,19.212558
min,1.000000,40.000000,4.000000,0.000000,10.000000
25%,3.000000,54.000000,5.000000,2.000000,40.000000
50%,5.000000,69.000000,7.000000,5.000000,55.000000
75%,7.000000,85.000000,8.000000,7.000000,71.000000
max,9.000000,99.000000,9.000000,9.000000,100.000000


In [2]:
# Cell No. 2

# Convert the categorical attributes' values into numeric values
encoder = LabelEncoder()
data['Extracurricular Activities'] = encoder.fit_transform(
                                        data['Extracurricular Activities'])

# Get the independent (input) and dependent (output) features
X = data.drop(columns=['Performance Index'])
y = data['Performance Index']

# Split the dataset into training and testing sets
test_set_ratio = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, y,
                        test_size=test_set_ratio, random_state=42)

# Convert the Pandas objects (dataframes & series) to Numpy arrays
x_train = X_train.to_numpy()
x_test = X_test.to_numpy()
y_train = Y_train.to_numpy()
y_test = Y_test.to_numpy()

# Normalize the features
scaler = MinMaxScaler(feature_range = (0,1))
x_train = scaler.fit_transform(x_train)
y_train = scaler.fit_transform(y_train.reshape(-1,1))
x_test = scaler.fit_transform(x_test)
y_test = scaler.fit_transform(y_test.reshape(-1,1))

# In the above lines, y-arrays reshaped from 1D to 2D: from (n,) to (n,1)
# Reshaping made to satisfy the requirement of fit_transform()

print("Total number of instances in the dataset:", data.shape[0])
print("Total number of training instances:", x_train.shape[0])
print("Total number of test instances:", x_test.shape[0])

Total number of instances in the dataset: 10000
Total number of training instances: 8000
Total number of test instances: 2000


In [3]:
# Cell No. 3

# Use sk-learn's SDGRegressor for model tarining and prediction making

from sklearn.linear_model import SGDRegressor

LR = 0.01            # learning rate (alpha or eta)
maxit = 1000         # maximum number of iterations
tol = 0.001          # tolerence for the training error (cost function)

sdg_regressor = SGDRegressor(penalty=None, max_iter=maxit, tol=tol,
        learning_rate='constant', eta0=LR)
sdg_regressor.fit(x_train, y_train.reshape(-1))

#In the above line, y_train used reshaped from 2D to 1D: from (n,1) to (n,)
# Reshaping made to satisfy the requirement of fit()

# Prediction for the train and test sets
# Output array of predict() here is 1D of the form (n,)
y_pred_train_norm = sdg_regressor.predict(x_train)
y_pred_test_norm = sdg_regressor.predict(x_test)

# Reshape y-arrays from 1D to 2D: from (n,) to (n,1)
# Reshaping made to satisfy the requirement of inverse_transform()
y_pred_train_norm = y_pred_train_norm.reshape(-1,1)
y_pred_test_norm = y_pred_test_norm.reshape(-1,1)

# Obtain and print model parameters
w = sdg_regressor.coef_
b = sdg_regressor.intercept_
print('Model parameters (using GD): \nSlope=', w, '\nIntercept=', b)

Model parameters (using GD): 
Slope= [0.25448347 0.66792045 0.00847233 0.02819312 0.02032719] 
Intercept= [0.01736959]


In [4]:
# Cell No. 4

# Inverse normalization
y_train = scaler.inverse_transform(y_train)
y_test = scaler.inverse_transform(y_test)
y_pred_train = scaler.inverse_transform(y_pred_train_norm)
y_pred_test = scaler.inverse_transform(y_pred_test_norm)

# Convert the arrays from 2D to 1D format that is (n,)
y_train = y_train.reshape(-1)
y_pred_train = y_pred_train.reshape(-1)
y_test = y_test.reshape(-1)
y_pred_test = y_pred_test.reshape(-1)

In [5]:
# Cell No. 5

#Calculate MAE and MSE for evalutaion of the model on the train & test sets

mae_train = mean_absolute_error(y_train, y_pred_train)
mse_train = mean_squared_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
mse_test = mean_squared_error(y_test, y_pred_test)

# Print MAE and MSE

print("MAE in prediction for train data:", mae_train)
print("MAE in prediction for test data:", mae_test)
print("")
print("MSE in prediction for train data:", mse_train)
print("MSE in prediction for test data:", mse_test)

MAE in prediction for train data: 1.611494913509565
MAE in prediction for test data: 1.6887050468253402

MSE in prediction for train data: 4.113767277798228
MSE in prediction for test data: 4.488384300621772


In [6]:
# Cell No. 6

# Compare the predicted and target values for the test set

from tabulate import tabulate

# Create a list of tuples with true and predicted values
compare = [(y_t, y_p) for y_t,y_p in zip(y_test[21:31],y_pred_test[21:31])]

# Define the column headings for the comparison table
headers = ["True Value", "Predicted Value"]

# Print the comparison table
print(tabulate(compare, headers, tablefmt = "grid", 
               numalign = "center", floatfmt = "0.2f"))

+--------------+-------------------+
|  True Value  |  Predicted Value  |
+==============+===================+
|    15.00     |       16.65       |
+--------------+-------------------+
|    38.00     |       39.07       |
+--------------+-------------------+
|    76.00     |       76.56       |
+--------------+-------------------+
|    84.00     |       83.95       |
+--------------+-------------------+
|    32.00     |       31.77       |
+--------------+-------------------+
|    30.00     |       29.40       |
+--------------+-------------------+
|    92.00     |       88.64       |
+--------------+-------------------+
|    52.00     |       51.60       |
+--------------+-------------------+
|    57.00     |       56.65       |
+--------------+-------------------+
|    60.00     |       60.35       |
+--------------+-------------------+


In [7]:
# Cell No. 7

# Print arrays sizes at different stages

# Print the shapes (dimensions) of arrays (intermediate/working stage) 

print("\n============================================")
print("Printing the shape (dimensions) of the arrays during working:")
print("Shape (dimensions) of train_pred_norm:" , y_pred_train_norm.shape)
print("Shape (dimensions) of test_pred_norm:" , y_pred_test_norm.shape)

print("\n============================================")
print("Printing the shape (dimensions) of the arrays after reshaping:")
print("Shape (dimensions) of y_train after reshaping:" , y_train.shape)
print("Shape (dimensions) of train_pred after reshaping:", y_pred_train.shape)
print("Shape (dimensions) of y_test after reshaping:" , y_test.shape)
print("Shape (dimensions) of test_pred after reshaping:" , y_pred_test.shape)


Printing the shape (dimensions) of the arrays during working:
Shape (dimensions) of train_pred_norm: (8000, 1)
Shape (dimensions) of test_pred_norm: (2000, 1)

Printing the shape (dimensions) of the arrays after reshaping:
Shape (dimensions) of y_train after reshaping: (8000,)
Shape (dimensions) of train_pred after reshaping: (8000,)
Shape (dimensions) of y_test after reshaping: (2000,)
Shape (dimensions) of test_pred after reshaping: (2000,)
